In [3]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/chenpindong/My_work/miwear-physionet-2025")

In [9]:
from helper_code import *
from team_code import extract_features, train_model
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

In [5]:
all_recodes = []
base_path = "/home/work/hdd3/yubo10/code15/code15_output/"
for exam_dir in os.listdir(base_path):
    records = find_records(os.path.join(base_path, exam_dir))
    all_recodes.extend([os.path.join(base_path, exam_dir, record) for record in records])

In [ ]:
code15_info = pd.read_csv("../dataset/code15_info.csv")
ptb_info = pd.read_csv("../dataset/ptb-xl_info.csv")
sami_info = pd.read_csv("../dataset/sami_info_info.csv")

In [29]:
all_recodes = code15_info['path'].values.tolist() + ptb_info['path'].values.tolist() + sami_info['path'].values.tolist()

In [32]:
num_records = len(all_recodes)
features = np.zeros((num_records, 6), dtype=np.float64)
labels = np.zeros(num_records, dtype=bool)
for i in range(num_records):
    record = all_recodes[i]
    features[i] = extract_features(record)
    labels[i] = load_label(record)

In [41]:
num_records

360107

In [33]:
# Define the parameters for the random forest classifier and regressor.
n_estimators = 12  # Number of trees in the forest.
max_leaf_nodes = 34  # Maximum number of leaf nodes in each tree.
random_state = 56  # Random state; set for reproducibility.

In [38]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cscore = []
aurocs = []
for train_idx, test_idx in kfold.split(features, labels):
    # Fit the model.
    train_x = features[train_idx]
    test_x = features[test_idx]

    train_y = labels[train_idx]
    test_y = labels[test_idx]

    model = RandomForestClassifier(
    n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes, random_state=random_state).fit(train_x, train_y)

    binary_output = model.predict(test_x)
    probability_output = model.predict_proba(test_x)[:,1]

    challenge_score = compute_challenge_score(test_y, probability_output)
    auroc, auprc = compute_auc(test_y, probability_output)
    accuracy = compute_accuracy(test_y, binary_output)
    f_measure = compute_f_measure(test_y, binary_output)

    print(f"challenge_score : {challenge_score}, auroc: {auroc} accuracy: {accuracy}, f_measure {f_measure}")
    cscore.append(challenge_score)
    aurocs.append(auroc)

challenge_score : 0.11958762886597939, auroc: 0.7271443428991005 accuracy: 0.9797978395490267, f_measure 0.0
challenge_score : 0.1346153846153846, auroc: 0.7351722526226716 accuracy: 0.9797839549026687, f_measure 0.0
challenge_score : 0.12164948453608247, auroc: 0.7197734265102212 accuracy: 0.9797975590452784, f_measure 0.0
challenge_score : 0.10927835051546392, auroc: 0.7333529927333752 accuracy: 0.9797975590452784, f_measure 0.0
challenge_score : 0.09828178694158075, auroc: 0.7278125773994527 accuracy: 0.9797975590452784, f_measure 0.0


In [40]:
print(np.mean(cscore))
print(np.mean(challenge_score))
print(np.mean(aurocs))

0.11668252709489824
0.09828178694158075
0.7286511184329643


In [ ]:
challenge_score : 0.13058419243986255, auroc: 0.7457642384851573 accuracy: 0.9797978395490267, f_measure 0.0
challenge_score : 0.12225274725274725, auroc: 0.7531401707013906 accuracy: 0.9797839549026687, f_measure 0.0
challenge_score : 0.13058419243986255, auroc: 0.7527740109841362 accuracy: 0.9797975590452784, f_measure 0.0
challenge_score : 0.13195876288659794, auroc: 0.7537173115602436 accuracy: 0.9797975590452784, f_measure 0.0
challenge_score : 0.0027491408934707906, auroc: 0.46614339158301094 accuracy: 0.9797975590452784, f_measure 0.0

(360107, 6)

In [36]:
np.mean(cscore)

np.float64(0.1036258071825082)

In [ ]:
np.mean(cscore)

np.float64(0.6943078246627877)

In [14]:


probability_output


np.float64(0.014900112130910767)